In [25]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [26]:
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Questions☝️

In this project, I will address this broad question: **Which "kinds" of players are most likely to contribute a large amount of data?**

Specifically, my question is: **Can a user's age and experience predict the average duration of their sessions?**

In [27]:
# converting the columns 'experience' and 'gender' in players.csv to factors
players <- players |> mutate(across(c("experience", "gender"), as.factor))

# rename certain columns so that all follow the same convention
players <- players |> rename(age = Age) |> rename(hashed_email = hashedEmail)
sessions <- sessions |> rename(hashed_email = hashedEmail)

In [29]:
# reporting the means for quantitative variables in players.csv,
# which includes 'played_hours' and 'age'

mean_hours <- mean(players$played_hours, na.rm = TRUE)
mean_age <- mean(players$age, na.rm = TRUE)

mean_tbl <- tibble(
    a = mean_hours;
    b = mean_age;
)

mean_tbl

ERROR: Error in parse(text = x, srcfile = src): <text>:8:19: unexpected ';'
7: mean_tbl <- tibble(
8:     a = mean_hours;
                     ^
